# **UsingGridSearchCV in SVD , NMF, SVD++**

In [ ]:
import numpy as np
import pandas as pd
from surprise import SVD
from surprise import Dataset , Reader
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import NMF
from surprise import SVDpp

In [ ]:
ratings=pd.read_csv('/content/drive/My Drive/Dataset/Ratings .csv')
movies=pd.read_csv('/content/drive/My Drive/Dataset/movies.csv')
userdata=pd.read_csv('/content/drive/My Drive/Dataset/user_data.csv')

In [ ]:
movies=pd.read_csv('/content/drive/My Drive/Dataset/movies.csv')
movies

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [ ]:
ratings=pd.read_csv('/content/drive/My Drive/Dataset/Ratings .csv')
ratings

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [ ]:
userdata=pd.read_csv('/content/drive/My Drive/Dataset/user_data.csv')
userdata

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,2460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,1060


In [ ]:
df=ratings.merge(userdata, on='UserID',how='left').merge(movies, on='MovieID',how='left')
df

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,M,25,6,11106,Weekend at Bernie's (1989),Comedy
1000205,6040,1094,5,956704887,M,25,6,11106,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,956704746,M,25,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,956715648,M,25,6,11106,Sophie's Choice (1982),Drama


In [ ]:
df.shape

(1000209, 10)

In [ ]:
df=df.drop(columns='Timestamp')
df

,UserID,MovieID,Rating,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,F,1,10,48067,My Fair Lady (1964),Musical|Romance
3,1,3408,4,F,1,10,48067,Erin Brockovich (2000),Drama
4,1,2355,5,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,M,25,6,11106,Weekend at Bernie's (1989),Comedy
1000205,6040,1094,5,M,25,6,11106,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,M,25,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,M,25,6,11106,Sophie's Choice (1982),Drama


In [ ]:
df.shape

(1000209, 9)

In [ ]:
!pip install surprise

     |████████████████████████████████| 6.5MB 2.7MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1675738 sha256=952dc8f3dd4bcc035447a512cc39137aad415648ecddfab50f75648abf903c13
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [ ]:
#from collections import defaultdict

from surprise import SVD
from surprise import Dataset

In [ ]:
# First train an SVD algorithm on the movielens dataset.
from surprise import Reader
reader = Reader()
data = Dataset.load_from_df(df[['UserID', 'MovieID', 'Rating']], reader)
trainset = data.build_full_trainset()
#algo = SVD()
#algo.fit(trainset)

## **SVD**

In [ ]:
from surprise import SVD
from surprise import Dataset , Reader
from surprise.model_selection import GridSearchCV

data = Dataset.load_from_df(df[['UserID', 'MovieID', 'Rating']], reader)

param_grid = {
    "n_epochs": [5, 10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print("RMSE:",gs.best_score["rmse"])
print("MAE:",gs.best_score["mae"])

print("RMSE Best Params:",gs.best_params["rmse"])
print("MAE Best Params:",gs.best_params["mae"])


RMSE: 0.9294490459208742
MAE: 0.744212676082196
RMSE Best Params: {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
MAE Best Params: {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [ ]:
alg = gs.best_estimator['rmse']
alg.fit(data.build_full_trainset())
print(alg)

In [ ]:
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import Dataset , Reader

reader = Reader()

data = Dataset.load_from_df(df[['UserID', 'MovieID', 'Rating']], reader)

trainset, testset = train_test_split(data, test_size=.25)

algo=SVD(n_epochs=10, reg_pu=0.005, reg_qi=0.06)
predictions=algo.fit(trainset).test(testset)
accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 0.8988
MAE:  0.7107


0.7106940766818919

## **NMF**

In [ ]:
from surprise import NMF
from surprise import Dataset , Reader
from surprise.model_selection import GridSearchCV

data = Dataset.load_from_df(df[['UserID', 'MovieID', 'Rating']], reader)

param_grid = {
    "n_epochs": [5, 10],
    "reg_pu": [0.002, 0.005],
    "reg_qi": [0.04, 0.06]
}
gs = GridSearchCV(NMF, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print("RMSE:",gs.best_score["rmse"])
print("MAE:",gs.best_score["mae"])

print("RMSE Best Params:",gs.best_params["rmse"])
print("MAE Best Params:",gs.best_params["mae"])


RMSE: 0.9739949354094803
MAE: 0.7520528801459495
RMSE Best Params: {'n_epochs': 10, 'reg_pu': 0.005, 'reg_qi': 0.06}
MAE Best Params: {'n_epochs': 10, 'reg_pu': 0.005, 'reg_qi': 0.06}


In [ ]:
from surprise import NMF
from surprise.model_selection import train_test_split
from surprise import accuracy
trainset, testset = train_test_split(data, test_size=.25)

algo=NMF(n_epochs=10, reg_pu=0.005, reg_qi=0.06)
predictions=algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

RMSE: 0.9780


0.9779524432492549

In [ ]:
from surprise import NMF
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import Dataset , Reader

reader = Reader()

data = Dataset.load_from_df(df[['UserID', 'MovieID', 'Rating']], reader)

trainset, testset = train_test_split(data, test_size=.25)

algo=NMF(n_epochs=10, reg_pu=0.005, reg_qi=0.06)
predictions=algo.fit(trainset).test(testset)
accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 0.9794
MAE:  0.7544


0.7543552331161085

## **SVD++ (SVDpp)**

In [ ]:
from surprise import SVDpp
from surprise import Dataset , Reader
from surprise.model_selection import GridSearchCV

data = Dataset.load_from_df(df[['UserID', 'MovieID', 'Rating']], reader)

param_grid = {
    "n_epochs": [5, 10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6]
}
gs = GridSearchCV(SVDpp, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print("RMSE:",gs.best_score["rmse"])
print("MAE:",gs.best_score["mae"])

print("RMSE Best Params:",gs.best_params["rmse"])
print("MAE Best Params:",gs.best_params["mae"])
alg = gs.best_estimator['rmse']
alg.fit(data.build_full_trainset())
print(alg)

RMSE: 0.9294032210980075
MAE: 0.7440164508886827
RMSE Best Params: {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
MAE Best Params: {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [ ]:
from surprise.model_selection import train_test_split
from surprise import accuracy
trainset, testset = train_test_split(data, test_size=.25)

algo=SVDpp(n_epochs=10, lr_all=0.005, reg_all=0.4)
predictions=algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

RMSE: 0.9290


0.9289893135615476

In [ ]:
from surprise import SVDpp
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import Dataset , Reader

reader = Reader()

data = Dataset.load_from_df(df[['UserID', 'MovieID', 'Rating']], reader)

trainset, testset = train_test_split(data, test_size=.25)

algo=SVDpp(n_epochs=10, reg_pu=0.005, reg_qi=0.06)
predictions=algo.fit(trainset).test(testset)
accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 0.8813
MAE:  0.6943


0.6942869348714246

# **Results**

**SVD**

*   RMSE: 0.9294490459208742
*   MAE: 0.744212676082196

With best parameters are{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}

After performing train test split on best parametrs

*   RMSE: 0.8988
*   MAE:  0.7107



**NMF**

*   RMSE: 0.9739949354094803
*   MAE: 0.7520528801459495

With best parameters are{'n_epochs': 10, 'reg_pu': 0.005, 'reg_qi': 0.06}

After performing train test split on best parametrs

*   RMSE: 0.9794
*   MAE:  0.7544


**SVDpp**

*   RMSE: 0.9294032210980075
*   MAE: 0.7440164508886827

With best parameters are{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}

After performing train test split on best parametrs

*   RMSE: 0.8813
*   MAE:  0.6943

So, From above all the results the best RMSE and MAE is given by SVDpp RMSE: 0.8813 ,MAE:  0.6943.


